# Pyspark session & Dataset loading

In [1]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel  
import time
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-memory 6g pyspark-shell'

spark = SparkSession.builder.master("local[*]").appName("UKHousingPriceRegression").getOrCreate()
print("SparkSession creat!")

path = "/kaggle/input/uk-housing-prices-paid/price_paid_records.csv"

start = time.time()

df = spark.read.csv(path, header=True, inferSchema=True)
df = df.repartition(100)  
# df.persist(StorageLevel.DISK_ONLY)  

end = time.time()
print('Timpul de incarcare a setului de date:', end - start, 'seconds')

#df.printSchema()
#df.show(5)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/18 08:43:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession creat!


Timpul de incarcare a setului de date: 42.777597427368164 seconds


# Preprocessing

In [2]:
from pyspark.sql.functions import split, col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

start = time.time()

# Eliminarea coloanelor irelevante
columns_to_drop = ['Duration', 'Transaction unique identifier', 'PPDCategory Type', 'Record Status - monthly file only']
for column in columns_to_drop:
    df = df.drop(column)

if 'Date of Transfer' in df.columns:
    df = df.withColumn('Year', split(col('Date of Transfer'), '-').getItem(0).cast('int'))
    df = df.withColumn('Month', split(col('Date of Transfer'), '-').getItem(1).cast('int'))
    df = df.withColumn('Day', split(split(col('Date of Transfer'), '-').getItem(2), ' ').getItem(0).cast('int'))
else:
    print("Coloana 'Date of Transfer' a fost deja descompusa .")
# Eliminarea inregistrarilor cu Property Type = 'O'
df = df.filter(df['Property Type'] != 'O')

# Aplicarea Label Encoding pe coloane categorice
categorical_cols = ['Property Type', 'Old/New', 'Town/City', 'District', 'County']
indexers = [
    StringIndexer(inputCol=col, outputCol=col + " LE", handleInvalid="keep")
    for col in categorical_cols
]


pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

end = time.time()
print('Time taken for full preprocessing:', round(end - start, 2), 'seconds')


# df.select('Price', 'Property Type', 'Year', 'Month', 'Day', 'Old/New LE', 'Town/City LE', 'District LE', 'County LE', 'Property Type LE').show(5)

Time taken for full preprocessing: 187.65 seconds


In [5]:
from pyspark.ml.feature import VectorAssembler

features = ['Year', 'Month', 'Day', 'Old/New LE', 'Town/City LE', 'District LE', 'County LE', 'Property Type LE']

# Asamblarea într-un vector
assembler = VectorAssembler(inputCols=features, outputCol="features")
df_prepared = assembler.transform(df)

# Impartire dataset pe df_prepared
train_data, test_data = df_prepared.randomSplit([0.7, 0.3], seed=42)

# Random Forest Regressor

In [5]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import time

# Definire model 
rf = RandomForestRegressor(
    featuresCol="features", 
    labelCol="Price", 
    numTrees=20,       
    maxDepth=5,        
    maxBins=1500,        
    seed=42
)

# Antrenare model
start = time.time()
model = rf.fit(train_data)
end = time.time()

# Prezicere pe test
predictions = model.transform(test_data)


feature_names = assembler.getInputCols()
importances = model.featureImportances

# Afișare importance feature cu PySpark
for i, score in enumerate(importances):
    print(f"{feature_names[i]}: {score:.4f}")
    
# Evaluare
evaluator_rmse = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="r2")

rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print(f"Random Forest - Time: {end - start:.2f}s | RMSE: {rmse:.2f} | R²: {r2:.2f}")


25/06/16 16:45:28 WARN MemoryStore: Not enough space to cache rdd_159_96 in memory! (computed 29.6 MiB so far)
25/06/16 16:45:28 WARN BlockManager: Persisting block rdd_159_96 to disk instead.
25/06/16 16:45:28 WARN MemoryStore: Not enough space to cache rdd_159_97 in memory! (computed 5.5 MiB so far)
25/06/16 16:45:28 WARN BlockManager: Persisting block rdd_159_97 to disk instead.
25/06/16 16:45:28 WARN MemoryStore: Not enough space to cache rdd_159_99 in memory! (computed 12.5 MiB so far)
25/06/16 16:45:28 WARN BlockManager: Persisting block rdd_159_99 to disk instead.
25/06/16 16:48:02 WARN DAGScheduler: Broadcasting large task binary with size 1018.0 KiB            


Year: 0.2949
Month: 0.0000
Day: 0.0001
Old/New LE: 0.0005
Town/City LE: 0.2211
District LE: 0.2569
County LE: 0.0541
Property Type LE: 0.1723


Random Forest - Time: 663.98s | RMSE: 166177.37 | R²: 0.40


# Gradient Boosting Regressor

In [6]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import time

start = time.time()

# Definirea modelului
gbt = GBTRegressor(featuresCol="features", 
                   labelCol="Price", 
                   maxBins=1500, 
                   maxIter=10, 
                   maxDepth=3, 
                   stepSize=0.1, 
                   subsamplingRate=0.7, 
                   seed=42)

# Antrenare model
model = gbt.fit(train_data)

# Predictii
predictions = model.transform(test_data)

end = time.time()

# Evaluare
evaluator_rmse = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="r2")

rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print(f"GBTRegressor - Time: {end - start:.2f}s | RMSE: {rmse:.2f} | R²: {r2:.2f}")


GBTRegressor - Time: 650.75s | RMSE: 167127.40 | R²: 0.40


# Linear Regression

In [6]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import time

evaluator_rmse = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="Price", predictionCol="prediction", metricName="r2")

start_time = time.time()

lr = LinearRegression(featuresCol="features", labelCol="Price")

# Antrenare
train_start = time.time()
lr_model = lr.fit(train_data)
train_end = time.time()

# Predictie
predict_start = time.time()
predictions = lr_model.transform(test_data)
predict_end = time.time()

# Evaluare
rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

end_time = time.time()

print(f" Timp antrenare LR: {train_end - train_start:.2f} sec")
print(f" Timp predicție: {predict_end - predict_start:.2f} sec")
print(f" Timp total execuție: {end_time - start_time:.2f} sec")
print(f" RMSE: {rmse:.2f}")
print(f" R2: {r2:.2f}")


25/06/17 14:51:05 WARN Instrumentation: [be764961] regParam is zero, which might cause numerical instability and overfitting.


 Timp antrenare LR: 317.53 sec
 Timp predicție: 0.07 sec
 Timp total execuție: 535.30 sec
 RMSE: 201273.43
 R2: 0.12
